# Introduction
This notebook goes through multi-processing of NLP for aiming to analyse the dependency of David Bowie with the Media's

Specifically, we'll be walking through:

 - **Getting the data** - in this case, we'll be get the data from a dataset in .csv
 - **Cleaning the data** - we will walk through popular text pre-processing techniques
 - **Organizing the data** - we will organize the cleaned data into a way that is easy to input into other algorithms

The output of this notebook will be clean, organized data in two standard text formats:

  - Corpus - a collection of text
  - Document-Term Matrix - word counts in matrix format

In each formats, we'll divide in 3 category:

  - The press - the magazine from the text
  - The year of publish - The year of the interview
  - The album - Each interview is linked with an album
  - Author ?

### Problem Statement
:WIP

# Get the data

Luckily, we already have a raw csv, with all our data ready. I have to import the csv, and transform them in dataframe
for each categories.

As we decided before we have to divide the dataset in 3 high-level category.

In [5]:
import pandas as pd
import pickle

file = r'data/bowie_txt_analysis.csv'
df = pd.read_csv(file)


# Pickle all the data
all_text = open('./data/pickle/all_text.pkl', 'wb')
pickle.dump(df, all_text)

# Create separate df for each category
df_press = df[['press', 'texte']]
df_album = df[['album', 'texte']]
df_author = df[['author', 'texte']]

df_album = df.groupby(['album'])['texte'].apply(' '.join).reset_index()
df_author = df.groupby(['author'])['texte'].apply(' '.join).reset_index()
df_press = df.groupby(['press'])['texte'].apply(' '.join).reset_index()

# Pickle all the category
album_pkl = open('./data/pickle/album.pkl', 'wb')
pickle.dump(df_album, album_pkl)

author_pkl = open('./data/pickle/author.pkl', 'wb')
pickle.dump(df_author, author_pkl)

press_pkl = open('./data/pickle/press.pkl', 'wb')
pickle.dump(df_press, press_pkl)

# Cleaning the data

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. With text data, there are some common data cleaning techniques, which are also known as text pre-processing techniques.

With text data, this cleaning process can go on forever. There's always an exception to every cleaning step. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate. Here are a bunch of things you can do to clean your data. We're going to execute just the common cleaning steps here and the rest can be done at a later point to improve our results.

**Common data cleaning steps on all text**:

- Make text all lower case
- Remove punctuation
- Remove numerical values
- Remove common non-sensical text (/n)
- Tokenize text
- Remove stop words


**More data cleaning steps after tokenization**:
- Stemming / lemmatization
- Parts of speech tagging
- Create bi-grams or tri-grams
- Deal with typos
- And more...


In [6]:
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round1 = lambda x: clean_text_round1(x)
round2 = lambda x: clean_text_round2(x)


df_press['texte'] = pd.DataFrame(df_press.texte.apply(round1))
df_press['texte'] = pd.DataFrame(df_press.texte.apply(round2))

df_album['texte'] = pd.DataFrame(df_album.texte.apply(round1))
df_album['texte'] = pd.DataFrame(df_album.texte.apply(round2))

df_author['texte'] = pd.DataFrame(df_author.texte.apply(round1))
df_author['texte'] = pd.DataFrame(df_author.texte.apply(round2))


# Organizing The Data
I mentioned earlier that the output of this notebook will be clean, organized data in two standard text formats:

 - **Corpus** - a collection of text
 - **Document**-Term Matrix - word counts in matrix format

## Document-Term Matrix
For many of the techniques we'll be using in future notebooks, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words. We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.

In addition, with CountVectorizer, we can remove stop words. Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt


cv = CountVectorizer(stop_words='english')


# DTM for press
data_press_cv = cv.fit_transform(df_press['texte'])
data_press_dtm = pd.DataFrame(data_press_cv.toarray(), columns=cv.get_feature_names())
data_press_dtm.index = df_press.press
press_dtm_pkl = open('./data/pickle/press_dtm.pkl', 'wb')
pickle.dump(data_press_dtm, press_dtm_pkl)

# DTM for author
data_author_cv = cv.fit_transform(df_author['texte'])
data_author_dtm = pd.DataFrame(data_author_cv.toarray(), columns=cv.get_feature_names())
data_author_dtm.index = df_author.author
author_dtm_pkl = open('./data/pickle/author_dtm.pkl', 'wb')
pickle.dump(data_author_dtm, author_dtm_pkl)

# DTM for album
data_album_cv = cv.fit_transform(df_album['texte'])
data_album_dtm = pd.DataFrame(data_album_cv.toarray(), columns=cv.get_feature_names())
data_album_dtm.index = df_album.album
album_dtm_pkl = open('./data/pickle/album_dtm.pkl', 'wb')
pickle.dump(data_album_dtm, album_dtm_pkl)



# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object
import pickle
pickle.dump(cv, open("./data/pickle/cv.pkl", "wb"))